### **Zadanie 1: Parent Document Retrieval z ChromaDB i embeddingami Sentence Transformers**

#### Cel:

Zaimplementuj system wyszukiwania ofert mieszkaniowych, który:

* indeksuje mniejsze fragmenty ogłoszeń (tzw. **chunki**),
* ale podczas wyszukiwania zwraca **pełny oryginalny opis oferty** (czyli **dokument nadrzędny / parent document**), z którego pochodzi trafienie.

---

### Dane:

Użyj pliku CSV: `adresowo_offers_all_categories.csv`
Źródło: Kaggle (np. poprzez `kagglehub` lub ręczne pobranie)
**Kolumna kluczowa:** `description` – zawiera pełny tekst ogłoszenia

---

### 🛠️ Kroki do wykonania:

1. **Wczytaj dane**

   * Załaduj dane z pliku CSV za pomocą `pandas`
   * Usuń wiersze, w których `description` jest pusty (`NaN`)

2. **Chunkowanie opisów**

   * Podziel każdy opis (`description`) na mniejsze fragmenty

3. **Embeddingi**

   * Użyj modelu `sentence-transformers/all-MiniLM-L6-v2` do wygenerowania embeddingów fragmentów
   * Przykład biblioteki: `sentence-transformers`

4. **Indeksowanie**

   * Zbuduj bazę ChromaDB z embeddingami fragmentów
   * Zapewnij, że zachowane jest powiązanie z dokumentem nadrzędnym

5. **Wyszukiwanie**

   * Zaimplementuj funkcję `search(query)`, która:

     * oblicza embedding zapytania,
     * znajduje top-k najbardziej podobnych fragmentów w indeksie,
     * zwraca **unikalne** pełne opisy (`parent_documents`) z których pochodzą trafienia



In [ ]:
!pip install kagglehub pandas

In [3]:
import pandas as pd
import kagglehub

rag_worksop = kagglehub.dataset_download('martininf1n1ty/rag-workshop')


# Wczytaj dane z pliku CSV
df = pd.read_csv(f'{rag_worksop}/adresowo_offers_all_categories.csv')

# Usuń wiersze z pustymi wartościami w kolumnie 'description'
df = df.dropna(subset=['description'])

# Opcjonalnie: wyświetl informacje o załadowanych danych
print(f"Załadowano {len(df)} wierszy po usunięciu pustych opisów")
print(df.head())


Załadowano 89 wierszy po usunięciu pustych opisów
              category                                          offer_url  \
0  mieszkania_sprzedaz  https://adresowo.pl/o/mieszkanie-warszawa-wola...   
1  mieszkania_sprzedaz  https://adresowo.pl/o/mieszkanie-warszawa-targ...   
2  mieszkania_sprzedaz  https://adresowo.pl/o/mieszkanie-warszawa-bemo...   
3  mieszkania_sprzedaz  https://adresowo.pl/o/mieszkanie-warszawa-ocho...   
4  mieszkania_sprzedaz  https://adresowo.pl/o/mieszkanie-warszawa-ursu...   

   price  currency  num_images  \
0    NaN       NaN           4   
1    NaN       NaN          14   
2    NaN       NaN          20   
3    NaN       NaN           6   
4    NaN       NaN          14   

                                      street_address  \
0    ul. Księcia Janusza, Warszawa Wola, mazowieckie   
1       ul. Toruńska, Warszawa Targówek, mazowieckie   
2  ul. Aleksandra Świętochowskiego, Warszawa Bemo...   
3  ul. Bitwy Warszawskiej 1920 r., Warszawa Ochot...   
4 

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [2]:
!pip install -U langchain langchain-core langchain-community langchain-openai langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 11.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.72
    Uninstalling langchain-core-0.3.72:
      Successfully uninstalled langchain-core-0.3.72
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-te

In [4]:
import pandas as pd
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

# ----------------------------------------
# 1. Dane → Document[]
# ----------------------------------------
docs = [
    Document(
        page_content=d,
        metadata={"row_id": i}
    )
    for i, d in enumerate(df["description"].dropna().head(2))
]

print(f"Załadowano {len(docs)} dokumentów wejściowych.")

# ----------------------------------------
# 2. Splitter dokumentów
# ----------------------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,         # możesz dostosować
    chunk_overlap=80,
    add_start_index=True
)

split_docs = text_splitter.split_documents(docs)

2025-11-13 15:38:32.804687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763048313.062709      79 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763048313.133656      79 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Załadowano 2 dokumentów wejściowych.


In [13]:
!pip install sentence-transformers

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 17.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.3 MB/s eta 0:00:00a 0:00:01
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl (432 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf = HuggingFaceEmbeddings(
   model_name=model_name,
   model_kwargs={'device': 'cpu'},
   encode_kwargs={'normalize_embeddings': False}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
vector_store = InMemoryVectorStore(hf)
vector_store.add_documents(split_docs)

# ----------------------------------------
# 4. RETRIEVER – wyszukiwarka semantyczna
# ----------------------------------------
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

def search(query: str):
    results = retriever.invoke(query)
    return results

# ----------------------------------------
# 5. ZAPYTANIE
# ----------------------------------------
query = "czynności związane z przetwarzaniem danych klientów"

results = search(query)

# ----------------------------------------
# 6. WYNIKI
# ----------------------------------------
for i, doc in enumerate(results):
    print(f"=== WYNIK {i} ===")
    print(doc.page_content)
    print("METADATA:", doc.metadata)
    print()

=== WYNIK 0 ===
Na sprzedaż duże, jasne mieszkanie o powierzchni 54 m² z balkonem, piwnicą oraz miejscem parkingowym przed budynkiem, zlokalizowane w spokojnej i zielonej części Targówka Mieszkaniowego.
METADATA: {'row_id': 1, 'start_index': 0}

=== WYNIK 1 ===
Dzięki funkcjonalnemu układowi i braku ścian nośnych w części mieszkania, istnieje możliwość dowolnej aranżacji wnętrz - np. połączenia kuchni z salonem lub stworzenia przestronnej strefy dziennej. Na zdjęciach widoczne są również możliwe aranżacje wnętrz po remoncie.
METADATA: {'row_id': 1, 'start_index': 974}

=== WYNIK 2 ===
ROZKŁAD POMIESZCZEŃ:
• Pokój dzienny: ok. 15 m² - wyjście na balkon.
• Kuchnia (umeblowana): ok. 6 m².
• Łazienka: ok. 5 m².
• Sypialnia z szafą wnękową: ok. 13 m².
• Pokój dziecięcy z szafą wnękową: ok. 10 m².
• Przedpokój z szafą wnękową: ok. 5 m².
• Balkon: widok na zieleń i dużą, otwartą przestrzeń.
• Do mieszkania przynależy również piwnica.
METADATA: {'row_id': 1, 'start_index': 490}



In [1]:
!cat README.md > README.md